In [7]:
import json
import pandas as pd
from evaluate import load
bert_score = load("bertscore")
def BertScore(refs, preds):
    bert_score_res = bert_score.compute(predictions=[preds], references=[refs], model_type="microsoft/deberta-xlarge-mnli", lang="en")
    
    return bert_score_res

In [8]:
a = 'Manchester City defeated Manchester United 3-1 with a double from Phil Foden and a goal from Erling Haaland.'
b = 'Manchester City came from behind to beat Manchester United 3-1 in a thrilling Premier League match. Phil Foden scored a brace, including a stunning equalizer, and Erling Haaland added a third goal for City in stoppage time. Marcus Rashford had given United an early lead with a sensational strike, but City dominated the game and deservedly took all three points. The win moves City to within a point of leaders Liverpool, while United remain in sixth place.'
print(BertScore(b,a))

/home/xbr/anaconda3/envs/LLM/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/xbr/anaconda3/envs/LLM/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


{'precision': [0.8450139760971069], 'recall': [0.5974714756011963], 'f1': [0.7000024914741516], 'hashcode': 'microsoft/deberta-xlarge-mnli_L40_no-idf_version=0.3.12(hug_trans=4.39.3)'}


In [19]:
qwen1_5_files = ['xsum_qwen_no_limit_len_pretrained__ahxt__LiteLlama-460M-1T__trust_remote_code__True.jsonl',
                 'newsroom_qwen_no_limit_len_pretrained__ahxt__LiteLlama-460M-1T__trust_remote_code__True.jsonl',
                 'cnndm_qwen_no_limit_len_pretrained__ahxt__LiteLlama-460M-1T__trust_remote_code__True.jsonl',
                 'bbc2024_qwen_no_limit_len_pretrained__ahxt__LiteLlama-460M-1T__trust_remote_code__True.jsonl']

llama2_files = ['xsum_llama2_no_limit_len_pretrained__ahxt__LiteLlama-460M-1T__trust_remote_code__True.jsonl',
                'newsroom_llama2_no_limit_len_pretrained__ahxt__LiteLlama-460M-1T__trust_remote_code__True.jsonl',
                'cnndm_llama2_no_limit_len_pretrained__ahxt__LiteLlama-460M-1T__trust_remote_code__True.jsonl',
                'bbc2024_llama2_no_limit_len_pretrained__ahxt__LiteLlama-460M-1T__trust_remote_code__True.jsonl']

llama3_files = ['xsum_sample_500_0k5_1k5_llama3_70b_summary_no_len_limit.jsonl',
                'newsroom_sample_500_0k5_1k5_llama3_70b_summary_no_len_limit.jsonl',
                'cnndm_sample_500_0k5_1k5_llama3_70b_summary_no_len_limit.jsonl',
                'bbc2024_sample_500_0k5_1k5_llama3_70b_summary_no_len_limit.jsonl']

base_qwen_path = './qwen1_5/ahxt_LiteLlama-460M-1T'

base_llama2_path = './llama2/ahxt_LiteLlama-460M-1T'

base_llama3_path = '/home/xbr/LLM/summary_benchmark/dataset/sample_500_llama3_70b_summary/'

for i in range(len(qwen1_5_files)):
    name = qwen1_5_files[i].split('_')[0]
    print(name)
    
    f1 = base_qwen_path + '/' + qwen1_5_files[i]
    f2 = base_llama2_path + '/' + llama2_files[i]
    f3 = base_llama3_path + '/' + llama3_files[i]
    #use json to load f1 and f2
    qwen = json.load(open(f1))
    llama2 = json.load(open(f2))
    
    llama3 = json.load(open(f3))
    
    
    score = 0
    
    for j in range(len(qwen)):
        score+= BertScore(llama2[j]['target'], llama3[j]["qwen_reference_summary"])['f1'][0]
        
    print(score/len(qwen))
    
    
    

xsum
0.7546372020244598
newsroom
0.7415115525722503
cnndm
0.759126748085022
bbc2024
0.7524805088043213


In [9]:
#evaluate ChatGLM3
import re
from tqdm import tqdm
def clean_summary(completion):
        completion = completion[2:] if completion.startswith("\n\n") else completion
        completion = completion.replace('The news summary is: \"', '')
        completion_list = completion.split('\n')
        if(len(completion_list)<3):
            completion = completion_list[-1]
        else:
            completion = completion.split('\n')[2]
        candidate = completion
        complete_sentences = re.findall(r'[^.!?]*[.!?]', completion)
        # 将匹配到的完整句子连接成一个新的字符串
        completion = ''.join(complete_sentences)
        if(completion==''):
            completion = candidate
        return completion
    
    
qwen1_5_files = ['xsum_sample_500_0k5_1k5_qwen1.5_72b_summary_no_len_limit.jsonl',
                'newsroom_sample_500_0k5_1k5_qwen1.5_72b_summary_no_len_limit.jsonl',
                'cnndm_sample_500_0k5_1k5_qwen1.5_72b_summary_no_len_limit.jsonl',
                'bbc2024_sample_500_0k5_1k5_qwen1.5_72b_summary_no_len_limit.jsonl']
base_qwen1_5_path = '/home/xbr/LLM/summary_benchmark/dataset/sample_500_qwen72b_summary'


llama2_files = ['xsum_sample_500_0k5_1k5_llama2_70b_summary_no_len_limit.jsonl',
                'newsroom_sample_500_0k5_1k5_llama2_70b_summary_no_len_limit.jsonl',
                'cnndm_sample_500_0k5_1k5_llama2_70b_summary_no_len_limit.jsonl',
                'bbc2024_sample_500_0k5_1k5_llama2_70b_summary_no_len_limit.jsonl']
base_llama2_path = '/home/xbr/LLM/summary_benchmark/dataset/sample_500_llama2_70b_summary'
 

from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm3-6b", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm3-6b", trust_remote_code=True).half().cuda()
model = model.eval()

average_result = dict()
for i in range(len(llama2_files)):
    name = llama2_files[i].split('_')[0]
    print(name)
    f1 = base_llama2_path + '/' + llama2_files[i]
    #use json to load f1 
    llama2 = json.load(open(f1))
    
    
    result_list = list()
    result_name = llama2_files[i] + '_chatglm3.jsonl'

    average_score = 0
    for j in tqdm(range(len(llama2))):
        news = llama2[j]['article']
        reference = llama2[j]['qwen_reference_summary']
        
        response, history = model.chat(tokenizer, f'News: {news}\nSummarize the news in two sentences. Summary:', history=[],do_sample=False)
        response = clean_summary(response)
        
        #f1 bertscore
        score = BertScore(reference, response)['f1'][0]
        average_score += score
        
        tmp_dict = dict()
        tmp_dict['article'] = news
        tmp_dict['target'] = reference
        tmp_dict['filtered_resps'] = response
        tmp_dict['bertscore_f1'] = score
        
        result_list.append(tmp_dict)
         
        
    average_result[name] = average_score/len(llama2)
    
    #save result to jsonl
    with open('./llama2/chatglm3/'+result_name, 'w') as f:
        json.dump(result_list, f, indent=4)
    with open('./llama2/chatglm3/average.jsonl', 'w') as f:
        json.dump([average_result], f, indent=4)
    


Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

xsum


 80%|███████▉  | 399/500 [09:33<02:25,  1.44s/it]


KeyboardInterrupt: 

In [7]:
file = './qwen1_5/chatglm3/bbc2024_sample_500_0k5_1k5_qwen1.5_72b_summary_no_len_limit.jsonl_chatglm3.jsonl'

data = json.load(open(file))

score = 0
for i in data:
    score += i['bertscore_f1']
print(score/len(data))

0.725672722876072


In [8]:
#两两之间的bertscore相似度

qwen1_5_files = ['xsum_qwen_no_limit_len_pretrained__ahxt__LiteLlama-460M-1T__trust_remote_code__True.jsonl',
                 'newsroom_qwen_no_limit_len_pretrained__ahxt__LiteLlama-460M-1T__trust_remote_code__True.jsonl',
                 'cnndm_qwen_no_limit_len_pretrained__ahxt__LiteLlama-460M-1T__trust_remote_code__True.jsonl',
                 'bbc2024_qwen_no_limit_len_pretrained__ahxt__LiteLlama-460M-1T__trust_remote_code__True.jsonl']

llama2_files = ['xsum_llama2_no_limit_len_pretrained__ahxt__LiteLlama-460M-1T__trust_remote_code__True.jsonl',
                'newsroom_llama2_no_limit_len_pretrained__ahxt__LiteLlama-460M-1T__trust_remote_code__True.jsonl',
                'cnndm_llama2_no_limit_len_pretrained__ahxt__LiteLlama-460M-1T__trust_remote_code__True.jsonl',
                'bbc2024_llama2_no_limit_len_pretrained__ahxt__LiteLlama-460M-1T__trust_remote_code__True.jsonl']

llama3_files = ['xsum_sample_500_0k5_1k5_llama3_70b_summary_no_len_limit.jsonl',
                'newsroom_sample_500_0k5_1k5_llama3_70b_summary_no_len_limit.jsonl',
                'cnndm_sample_500_0k5_1k5_llama3_70b_summary_no_len_limit.jsonl',
                'bbc2024_sample_500_0k5_1k5_llama3_70b_summary_no_len_limit.jsonl']

qwen2_file = ['xsum_sample_500_0k5_1k5_qwen2_summary_no_len_limit.jsonl',
                'newsroom_sample_500_0k5_1k5_qwen2_summary_no_len_limit.jsonl',
                'cnndm_sample_500_0k5_1k5_qwen2_summary_no_len_limit.jsonl',
                'bbc2024_sample_500_0k5_1k5_qwen2_summary_no_len_limit.jsonl']


phi3_files = ['xsum_qwen_no_limit_len_pretrained__microsoft__Phi-3-mini-4k-instruct__trust_remote_code__True.jsonl',
              'newsroom_qwen_no_limit_len_pretrained__microsoft__Phi-3-mini-4k-instruct__trust_remote_code__True.jsonl',
              'cnndm_qwen_no_limit_len_pretrained__microsoft__Phi-3-mini-4k-instruct__trust_remote_code__True.jsonl',
              'bbc2024_qwen_no_limit_len_pretrained__microsoft__Phi-3-mini-4k-instruct__trust_remote_code__True.jsonl']

qwen2_7b_files = ['xsum_qwen_no_limit_len_pretrained__Qwen__Qwen2-7B__trust_remote_code__True.jsonl',
            'newsroom_qwen_no_limit_len_pretrained__Qwen__Qwen2-7B__trust_remote_code__True.jsonl',
            'cnndm_qwen_no_limit_len_pretrained__Qwen__Qwen2-7B__trust_remote_code__True.jsonl',
            'bbc2024_qwen_no_limit_len_pretrained__Qwen__Qwen2-7B__trust_remote_code__True.jsonl']

mistral_ins_files = ['xsum_qwen_no_limit_len_pretrained__mistralai__Mistral-7B-Instruct-v0.2__trust_remote_code__True.jsonl',
                    'newsroom_qwen_no_limit_len_pretrained__mistralai__Mistral-7B-Instruct-v0.2__trust_remote_code__True.jsonl',
                    'cnndm_qwen_no_limit_len_pretrained__mistralai__Mistral-7B-Instruct-v0.2__trust_remote_code__True.jsonl',
                    'bbc2024_qwen_no_limit_len_pretrained__mistralai__Mistral-7B-Instruct-v0.2__trust_remote_code__True.jsonl']

chatglm3_files = ['xsum_sample_500_0k5_1k5_qwen1.5_72b_summary_no_len_limit.jsonl_chatglm3.jsonl',
           'newsroom_sample_500_0k5_1k5_qwen1.5_72b_summary_no_len_limit.jsonl_chatglm3.jsonl',
           'cnndm_sample_500_0k5_1k5_qwen1.5_72b_summary_no_len_limit.jsonl_chatglm3.jsonl',
           'bbc2024_sample_500_0k5_1k5_qwen1.5_72b_summary_no_len_limit.jsonl_chatglm3.jsonl']


base_chatglm3_path = './qwen1_5/chatglm3'

base_mistral_ins_path = './qwen1_5/mistralai_Mistral-7B-Instruct-v0.2'

base_qwen2_7b__path = './qwen1_5/Qwen_Qwen2-7B'

base_phi3 = './qwen1_5/microsoft_Phi-3-mini-4k-instruct_new_version/'

base_qwen_path = './qwen1_5/ahxt_LiteLlama-460M-1T'

base_llama2_path = './llama2/ahxt_LiteLlama-460M-1T'

base_llama3_path = '/home/xbr/LLM/summary_benchmark/dataset/sample_500_llama3_70b_summary'

base_qwen2_path = '/home/xbr/LLM/summary_benchmark/dataset/sample_500_qwen2_72b_summary'


total_score = 0
for i in range(len(qwen1_5_files)):
    name = qwen1_5_files[i].split('_')[0]
    print(name)
    
    f1 = base_qwen_path + '/' + qwen1_5_files[i]
    f2 = base_llama2_path + '/' + llama2_files[i]
    f3 = base_llama3_path + '/' + llama3_files[i]
    #use json to load f1 and f2
    qwen = json.load(open(f1))
    llama2 = json.load(open(f2))
    
    llama3 = json.load(open(f3))
    qwen2  = json.load(open(base_qwen2_path + '/' + qwen2_file[i]))
    phi3 = json.load(open(base_phi3 + '/' + phi3_files[i]))
    qwen2_7b = json.load(open(base_qwen2_7b__path + '/' + qwen2_7b_files[i]))
    mistral_ins = json.load(open(base_mistral_ins_path + '/' + mistral_ins_files[i]))
    chatglm3 = json.load(open(base_chatglm3_path + '/' + chatglm3_files[i]))
    
    score = 0
    
    for j in range(len(qwen)):
        # score+= BertScore(chatglm3[j]["filtered_resps"], phi3[j]["filtered_resps"][0])['f1'][0]
        # score+= BertScore(qwen2_7b[j]["filtered_resps"][0], phi3[j]["filtered_resps"][0])['f1'][0]
        # score+= BertScore(qwen2[j]["qwen_reference_summary"],chatglm3[j]['filtered_resps'])['f1'][0]
         score+= BertScore(phi3[j]["filtered_resps"][0], qwen2[j]["qwen_reference_summary"])['f1'][0]
        # score+= BertScore(phi3[j]["filtered_resps"][0], qwen[j]['target'])['f1'][0]
        
    print(score/len(qwen))
    total_score+=score/len(qwen)
print("total ")
print((total_score/4)*100)
    
    
  #phi3
  #72.80, 73.47, 74.85, 100, 74.02, 74.51, 74.01, 72.52

xsum
0.7305820472240449
newsroom
0.710476155757904
cnndm
0.7347142081260681
bbc2024
0.7250446029901505
total 
72.5204253524542


In [22]:
import torch 
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline 
import transformers
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"


class MultiTokenEOSCriteria(transformers.StoppingCriteria):
    """Criteria to stop on the specified multi-token sequence."""

    def __init__(
        self,
        sequence: str,
        tokenizer: transformers.PreTrainedTokenizer,
        initial_decoder_input_length: int,
        batch_size: int,
    ) -> None:
        self.initial_decoder_input_length = initial_decoder_input_length
        self.done_tracker = [False] * batch_size
        self.sequence = sequence
        self.sequence_ids = tokenizer.encode(sequence, add_special_tokens=False)
        # print(sequence, self.sequence_ids)
        # we look back for 2 more tokens than it takes to encode our stop sequence
        # because tokenizers suck, and a model might generate `['\n', '\n']` but our `sequence` is `['\n\n']`
        # and we don't want to mistakenly not stop a generation because our
        # (string) stop sequence was output in a different tokenization

        # NOTE: there is a minor danger that this will end up looking back 2 tokens into the past, into the inputs to the model,
        # and stopping generation immediately as a result. With only 2 extra tokens of lookback, this risk is minimized
        # Additionally, in lookback_ids_batch we should prevent ever looking back into the inputs as described.
        self.sequence_id_len = len(self.sequence_ids) + 2
        self.tokenizer = tokenizer

    def __call__(self, input_ids, scores, **kwargs) -> bool:
        # For efficiency, we compare the last n tokens where n is the number of tokens in the stop_sequence
        lookback_ids_batch = input_ids[:, self.initial_decoder_input_length :]

        lookback_ids_batch = lookback_ids_batch[:, -self.sequence_id_len :]

        lookback_tokens_batch = self.tokenizer.batch_decode(lookback_ids_batch)

        for i, done in enumerate(self.done_tracker):
            if not done:
                self.done_tracker[i] = self.sequence in lookback_tokens_batch[i]
        return False not in self.done_tracker


def stop_sequences_criteria(
    tokenizer: transformers.PreTrainedTokenizer,
    stop_sequences: list[str],
    initial_decoder_input_length: int,
    batch_size: int,
) -> transformers.StoppingCriteriaList:
    return transformers.StoppingCriteriaList(
        [
            *[
                MultiTokenEOSCriteria(
                    sequence, tokenizer, initial_decoder_input_length, batch_size
                )
                for sequence in stop_sequences
            ],
        ]
    )



torch.random.manual_seed(0) 
model = AutoModelForCausalLM.from_pretrained( 
    "microsoft/Phi-3-mini-4k-instruct",  
    device_map="cuda",  
    torch_dtype="auto",  
    trust_remote_code=True,  
) 

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct") 
prompt = "News: Last updated on .From the section Premier League Pep Guardiola hailed Phil Foden as the \"best player in the Premier League right now\" after his second-half double ensured Manchester City came from behind to beat Manchester United at Etihad Stadium. In a game packed with international stars, it was two local boys who provided the key moments. The visitors had led through Marcus Rashford's sensational eighth-minute strike from 30 yards. However, in a contest City dominated, Foden took centre stage. There was an element of controversy over his second-half equaliser. United boss Erik ten Hag was booked for arguing his side should have had a free-kick in the City half when Rashford went down under Kyle Walker's challenge. Contact was minimal and within seconds Foden had curled a superb shot past Andre Onana. Foden then burst away from a static Casemiro to score City's second from Julian Alvarez's return pass before Erling Haaland rounded off the scoring in stoppage time, after the Norwegian earlier missed an open goal from barely three yards. \"It is the amount of games he is playing,\" Guardiola told Match of the Day about Foden. \"He was always a talented player but now he is more mature and understands more the game, especially defensively. He can play middle, right, make moments and cut inside, play in the left, scoring from the left. \"What can I say? He is the best player in the Premier League right now for the amount of things he does. Unbelievable.\" It was City's sixth win in seven meetings against United, who suffered their 11th Premier League defeat of the season. More importantly, the result means Pep Guardiola's side move to within a point of leaders Liverpool before next week's trip to Anfield. United remain in sixth in the table but are now 11 points behind fourth-placed Aston Villa and six behind Tottenham, in what could turn out to be a fifth Champions League spot, having played a game more. • None Premier League title race run-in: Will Liverpool, Manchester City or Arsenal come out on top? There was something fitting about Stockport-born Foden becoming the match winner. It seems a long time ago now that Guardiola was having to defend his treatment of a player he was accused of be holding back while contemporary Jadon Sancho was excelling at Borussia Dortmund having decided he would not wait to develop at City. The contrast in current fortunes for the two players could not be more marked, with United outcast Sancho back on loan at Dortmund. Foden's equaliser was City's 600th goal in this stadium under Guardiola and was a sublime effort, curled into the top corner out of Onana's reach after he had run across Victor Lindelof on the right edge of the United box. His second saw him burst into the area from the other side, collect a pass from Julian Alvarez and drill a low shot home, almost through the United keeper, who might have done better. Foden is now picking his moments to get involved in attacks and watching England boss Gareth Southgate must surely find a way of getting the 23-year-old, who has scored 18 goals this season, into his starting line-up at Euro 2024 alongside the likes of Harry Kane and Jude Bellingham. Haaland will have been pleased to get his name on the scoresheet given his incredible first-half miss when he somehow put Foden's knock-back over an open goal from three yards. \"He was disappointed, I was disappointed. I want him to score four or five goals every game like he did against Luton [in the FA Cup],\" said Guardiola. \"But I don't care. He can miss this one, it is the reaction. He is sad for 10 seconds and he can miss five more and is sad for 10 seconds in his mind but after that erase it and on to the next one. \"The great, great players I met, and I've been fortunate as a player and especially a manager, they have this incredible ability to forget in an instant. \"Tennis players, golf players, basketball players, when they miss, and everyone misses, they say 'OK', smile, be positive and go for the next. That defines the great players and he did it.\" • None How did you rate Manchester City's performance? Have your say here • None What did you make of Manchester United's display? Send us your views here Rashford on target but United well beaten In the build-up to the game, Rashford spoke extensively to Players' Tribune, detailing his journey through poverty to United's first team and underlining how much the club means to him. It was thought-provoking stuff and very personal. Yet, for a sizeable number of fans who have viewed Rashford's efforts this season through the prism of underperformance, a lack of goal threat and a body language of indifference, the words meant nothing. For them the general reaction was 'save it for the pitch'. To that end, it is quite possible Rashford will never score a better goal than his eighth-minute opener. Bruno Fernandes did well to hold off Ruben Dias to control Onana's booming kick downfield. Then, as Scott McTominay sprinted into the area, Fernandes had the vision and intelligence to wait for Rashford coming behind. He rolled a pass perfectly to the England striker, who did not have to break stride as he launched a shot into the City net off the underside of the bar. Rashford could easily have had another but, in attempting to control a bouncing ball as Fernandes lifted a pass towards him beyond a static home defence, he succeeded only in heading it into the ground, which allowed Kyle Walker to get back and snuff out the danger. That turned out to be United's last realistic opportunity as they attempted to repel wave after wave of City attacks. Ten Hag simply did not have the personnel to complete the task, as hard as his players tried. New co-owner Sir Jim Ratcliffe has said he wants to knock City off their perch. On this evidence, he is going to need a pretty long ladder to attempt that. • None Attempt missed. Casemiro (Manchester United) header from the centre of the box misses to the right. Assisted by Bruno Fernandes following a set piece situation. • None Substitution, Manchester City. Oscar Bobb replaces Phil Foden because of an injury. • None Goal! Manchester City 3, Manchester United 1. Erling Haaland (Manchester City) left footed shot from the right side of the box to the bottom left corner. Assisted by Rodri. • None Attempt missed. Kevin De Bruyne (Manchester City) right footed shot from outside the box is close, but misses to the right. Assisted by Phil Foden. • None Rodri (Manchester City) wins a free kick on the right wing. • None Goal! Manchester City 2, Manchester United 1. Phil Foden (Manchester City) left footed shot from the left side of the box to the bottom right corner. Assisted by Julián Álvarez. Navigate to the next page Navigate to the last page\nSummarize the news in two sentences. Summary:"

messages = [ 
    {"role": "system", "content": "You are a helpful AI assistant."}, 
    {"role": "user", "content": prompt}, 
   
] 



pipe = pipeline( 
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
) 

generation_args = { 
    "max_new_tokens": 256, 
    "return_full_text": False, 
    "temperature": 0.0, 
    "do_sample": False,
} 

output = pipe(messages, **generation_args) 
print(output[0]['generated_text'])
print(len(output[0]['generated_text'].split(' ')))

# inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False).to('cuda:0')

# stopping_criteria = stop_sequences_criteria(tokenizer, ['\n', '</s>'], inputs['input_ids'].shape[1], 1)

# outputs = model.generate(**inputs, max_length=2000, stopping_criteria=stopping_criteria, do_sample=False)
# text = tokenizer.batch_decode(outputs)[0]
# print(text)

/home/xbr/anaconda3/envs/LLM/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/xbr/anaconda3/envs/LLM/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 Manchester City's Phil Foden was hailed as the best player in the Premier League by Pep Guardiola after scoring two crucial goals against Manchester United, helping City secure a 3-1 victory at Etihad Stadium. The win moves City within a point of Liverpool for the title race, while United's performance highlighted the need for improvement under Erik ten Hag's management.
61


In [2]:
#pegasus

import pandas as pd
import numpy as np  
from tqdm import tqdm
import json
import torch
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM
torch.set_default_device("cuda:0")


doc = 'President Joe Biden’s mission to speed up the lackluster rollout of coronavirus vaccines. Biden signed a slew of executive orders this week aimed at accelerating the distribution of vaccines.'

model_name = 'google/pegasus-large'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


with torch.no_grad():
    batch = tokenizer(doc, truncation=True, padding="longest", return_tensors="pt")
    preds = model.generate(**batch)
    tgt_text = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    print(tgt_text[0])


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Biden signed a slew of executive orders this week aimed at accelerating the distribution of vaccines.


In [5]:
#qwen2 ins

from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-0.5B-Instruct",
    torch_dtype="auto",
    device_map="auto"
).to(device)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B-Instruct")


news = "Last updated on .From the section Premier League Pep Guardiola hailed Phil Foden as the \"best player in the Premier League right now\" after his second-half double ensured Manchester City came from behind to beat Manchester United at Etihad Stadium. In a game packed with international stars, it was two local boys who provided the key moments. The visitors had led through Marcus Rashford's sensational eighth-minute strike from 30 yards. However, in a contest City dominated, Foden took centre stage. There was an element of controversy over his second-half equaliser. United boss Erik ten Hag was booked for arguing his side should have had a free-kick in the City half when Rashford went down under Kyle Walker's challenge. Contact was minimal and within seconds Foden had curled a superb shot past Andre Onana. Foden then burst away from a static Casemiro to score City's second from Julian Alvarez's return pass before Erling Haaland rounded off the scoring in stoppage time, after the Norwegian earlier missed an open goal from barely three yards. \"It is the amount of games he is playing,\" Guardiola told Match of the Day about Foden. \"He was always a talented player but now he is more mature and understands more the game, especially defensively. He can play middle, right, make moments and cut inside, play in the left, scoring from the left. \"What can I say? He is the best player in the Premier League right now for the amount of things he does. Unbelievable.\" It was City's sixth win in seven meetings against United, who suffered their 11th Premier League defeat of the season. More importantly, the result means Pep Guardiola's side move to within a point of leaders Liverpool before next week's trip to Anfield. United remain in sixth in the table but are now 11 points behind fourth-placed Aston Villa and six behind Tottenham, in what could turn out to be a fifth Champions League spot, having played a game more. • None Premier League title race run-in: Will Liverpool, Manchester City or Arsenal come out on top? There was something fitting about Stockport-born Foden becoming the match winner. It seems a long time ago now that Guardiola was having to defend his treatment of a player he was accused of be holding back while contemporary Jadon Sancho was excelling at Borussia Dortmund having decided he would not wait to develop at City. The contrast in current fortunes for the two players could not be more marked, with United outcast Sancho back on loan at Dortmund. Foden's equaliser was City's 600th goal in this stadium under Guardiola and was a sublime effort, curled into the top corner out of Onana's reach after he had run across Victor Lindelof on the right edge of the United box. His second saw him burst into the area from the other side, collect a pass from Julian Alvarez and drill a low shot home, almost through the United keeper, who might have done better. Foden is now picking his moments to get involved in attacks and watching England boss Gareth Southgate must surely find a way of getting the 23-year-old, who has scored 18 goals this season, into his starting line-up at Euro 2024 alongside the likes of Harry Kane and Jude Bellingham. Haaland will have been pleased to get his name on the scoresheet given his incredible first-half miss when he somehow put Foden's knock-back over an open goal from three yards. \"He was disappointed, I was disappointed. I want him to score four or five goals every game like he did against Luton [in the FA Cup],\" said Guardiola. \"But I don't care. He can miss this one, it is the reaction. He is sad for 10 seconds and he can miss five more and is sad for 10 seconds in his mind but after that erase it and on to the next one. \"The great, great players I met, and I've been fortunate as a player and especially a manager, they have this incredible ability to forget in an instant. \"Tennis players, golf players, basketball players, when they miss, and everyone misses, they say 'OK', smile, be positive and go for the next. That defines the great players and he did it.\" • None How did you rate Manchester City's performance? Have your say here • None What did you make of Manchester United's display? Send us your views here Rashford on target but United well beaten In the build-up to the game, Rashford spoke extensively to Players' Tribune, detailing his journey through poverty to United's first team and underlining how much the club means to him. It was thought-provoking stuff and very personal. Yet, for a sizeable number of fans who have viewed Rashford's efforts this season through the prism of underperformance, a lack of goal threat and a body language of indifference, the words meant nothing. For them the general reaction was 'save it for the pitch'. To that end, it is quite possible Rashford will never score a better goal than his eighth-minute opener. Bruno Fernandes did well to hold off Ruben Dias to control Onana's booming kick downfield. Then, as Scott McTominay sprinted into the area, Fernandes had the vision and intelligence to wait for Rashford coming behind. He rolled a pass perfectly to the England striker, who did not have to break stride as he launched a shot into the City net off the underside of the bar. Rashford could easily have had another but, in attempting to control a bouncing ball as Fernandes lifted a pass towards him beyond a static home defence, he succeeded only in heading it into the ground, which allowed Kyle Walker to get back and snuff out the danger. That turned out to be United's last realistic opportunity as they attempted to repel wave after wave of City attacks. Ten Hag simply did not have the personnel to complete the task, as hard as his players tried. New co-owner Sir Jim Ratcliffe has said he wants to knock City off their perch. On this evidence, he is going to need a pretty long ladder to attempt that. • None Attempt missed. Casemiro (Manchester United) header from the centre of the box misses to the right. Assisted by Bruno Fernandes following a set piece situation. • None Substitution, Manchester City. Oscar Bobb replaces Phil Foden because of an injury. • None Goal! Manchester City 3, Manchester United 1. Erling Haaland (Manchester City) left footed shot from the right side of the box to the bottom left corner. Assisted by Rodri. • None Attempt missed. Kevin De Bruyne (Manchester City) right footed shot from outside the box is close, but misses to the right. Assisted by Phil Foden. • None Rodri (Manchester City) wins a free kick on the right wing. • None Goal! Manchester City 2, Manchester United 1. Phil Foden (Manchester City) left footed shot from the left side of the box to the bottom right corner. Assisted by Julián Álvarez. Navigate to the next page Navigate to the last page"
prompt = f'News: {news}\nSummarize the news in two sentences. Summary: '

messages = [
    {"role": "system", "content": "You are a helpful summary assistant. You can help users summarize news in two sentences."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512,
    do_sample=False,
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)
#Manchester City defeated Manchester United 3-1 with a crucial double from Phil Foden and a hat-trick from Erling Haaland.

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Manchester City defeated Manchester United 3-1 with a double from Phil Foden and a goal from Erling Haaland.
